## WHAT WE NEED TO DO
* FIGURE OUT HOW (or if) WE ARE NARROWING IT DOWN (2019 pulls close to 2,000 results)
* We need to figure out what info we want and make empty rows to port in the info into BOTH FOR OMDB AND TMDB
* MAKE ROW FOR RESPONSE = FALSE SO WE CAN COUNT ERROR

## Potential issues
* Filtering out awards ---> could sub in popularity instead (IMDB info has a detailed breakdown: https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset?select=IMDb+ratings.csv)

In [1]:
import pandas as pd
import requests
from config import omdb_key
from config import tmdb_key

In [2]:
# https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset

file_path = 'Resources/IMDb_movies.csv'

In [23]:
movie_titles_df = pd.read_csv(file_path)

print(movie_titles_df.columns)
movie_titles_df.head(2)

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')


/opt/anaconda3/envs/pythonUCI/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,12/26/06,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,"$2,250",NaN,NaN,NaN,7.0,7.0


# ### ANY OTHERS FILTERS/CHANGE????

In [4]:
movie_titles_clean = movie_titles_df[['imdb_title_id', 'title', 'year', 'date_published',
                                      'country', 'language', 'genre', 'reviews_from_critics']]
# remove 1 rouge string value in year
movie_titles_clean = movie_titles_clean[movie_titles_clean['year'] != 'TV Movie 2019']

# need to convert type to int & string
movie_titles_clean = movie_titles_clean.astype({'year': int, 'language': 'string',
                                               'country': 'string'})

# filter language based on Englsih and years 2014-2019
movie_titles_clean = movie_titles_clean[(movie_titles_clean.year > 2013) & (movie_titles_clean.year < 2020)]

### ANY OTHERS/CHANGE????
movie_titles_clean = movie_titles_clean[movie_titles_clean['language'].str.contains('English')]
movie_titles_clean = movie_titles_clean[movie_titles_clean['country'].str.contains('USA')]


In [18]:
print(movie_titles_clean.dtypes)
print(f"\nTHIS IS HOW MANY MOVIES: {movie_titles_clean.title.count()}")
movie_titles_clean.head()

imdb_title_id            object
title                    object
year                      int64
date_published           object
country                  string
language                 string
genre                    object
reviews_from_critics    float64
dtype: object

THIS IS HOW MANY MOVIES: 6055


,imdb_title_id,title,year,date_published,country,language,genre,reviews_from_critics
15675,tt0069049,L'altra faccia del vento,2018,11/2/18,"France, Iran, USA","English, German",Drama,98.0
42166,tt0326716,'77,2017,5/25/17,USA,English,"Comedy, Drama",4.0
42384,tt0331314,Bunyan and Babe,2017,1/12/17,"USA, India",English,"Animation, Adventure, Comedy",1.0
42695,tt0339736,The Evil Within,2017,8/30/17,USA,English,Horror,34.0
43366,tt0360556,Fahrenheit 451,2018,5/20/18,USA,English,"Drama, Sci-Fi, Thriller",74.0


In [21]:
movies_2019 = movie_titles_clean[movie_titles_clean.year == 2019]

print(f"THIS IS HOW MANY MOVIES: {movies_2019.title.count()}")

movies_2019.head()

THIS IS HOW MANY MOVIES: 860


,imdb_title_id,title,year,date_published,country,language,genre,reviews_from_critics
44619,tt0385887,Motherless Brooklyn - I segreti di una città,2019,11/7/19,USA,"English, French","Crime, Drama, Mystery",178.0
46723,tt0437086,Alita - Angelo della battaglia,2019,2/14/19,"USA, Japan, Canada","English, Spanish","Action, Adventure, Sci-Fi",368.0
47132,tt0448115,Shazam!,2019,4/3/19,"USA, Canada","English, Spanish","Action, Adventure, Comedy",379.0
49759,tt0783640,Era mio figlio,2019,4/2/20,USA,English,"Drama, War",44.0
49964,tt0800325,The Dirt,2019,3/22/19,USA,English,"Biography, Comedy, Drama",72.0


In [22]:
#TESTING DF

movies_test = movies_2019[0:15].reset_index()
movies_test.head()

,index,imdb_title_id,title,year,date_published,country,language,genre,reviews_from_critics
0,44619,tt0385887,Motherless Brooklyn - I segreti di una città,2019,11/7/19,USA,"English, French","Crime, Drama, Mystery",178.0
1,46723,tt0437086,Alita - Angelo della battaglia,2019,2/14/19,"USA, Japan, Canada","English, Spanish","Action, Adventure, Sci-Fi",368.0
2,47132,tt0448115,Shazam!,2019,4/3/19,"USA, Canada","English, Spanish","Action, Adventure, Comedy",379.0
3,49759,tt0783640,Era mio figlio,2019,4/2/20,USA,English,"Drama, War",44.0
4,49964,tt0800325,The Dirt,2019,3/22/19,USA,English,"Biography, Comedy, Drama",72.0


# Making the dataframe into a runable list

In [8]:
# change title name to have + instead of ' '
movies_to_run = movies_test['title'].str.replace(' ', '+')
movies_to_run

# pull movie titles into an array so you can run it
movies = movies_to_run.tolist()

# OMDB limits to 1,000 per day

# ***** FIGURE OUT WHAT WE NEED *****

In [9]:
# make URL
url_omdb = "http://www.omdbapi.com/?apikey="+ omdb_key + "&t="

response_omdb = []


for movie in movies:
    movie_data = requests.get(url_omdb + movie + "&y=2019").json()
    response_omdb.append(movie_data)


In [10]:
response_omdb

[{'Response': 'False', 'Error': 'Movie not found!'},
 {'Response': 'False', 'Error': 'Movie not found!'},
 {'Title': 'Shazam!',
  'Year': '2019',
  'Rated': 'PG-13',
  'Released': '05 Apr 2019',
  'Runtime': '132 min',
  'Genre': 'Action, Adventure, Comedy, Fantasy',
  'Director': 'David F. Sandberg',
  'Writer': 'Henry Gayden (screenplay by), Henry Gayden (story by), Darren Lemke (story by), Bill Parker (Shazam created by), C.C. Beck (Shazam created by)',
  'Actors': 'Zachary Levi, Mark Strong, Asher Angel, Jack Dylan Grazer',
  'Plot': 'A newly fostered young boy in search of his mother instead finds unexpected super powers and soon gains a powerful enemy.',
  'Language': 'English, Spanish',
  'Country': 'USA, Canada',
  'Awards': '21 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BOWZhZjE4NGQtODg5Ni00MjQ1LWJmMzAtNzQ2N2M1NzYzMDJkXkEyXkFqcGdeQXVyMjMwNDgzNjc@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.1/10'},
   {'Source': 'Rotte

# TMDB has no limits

In [11]:
url_tmdb_id = "https://api.themoviedb.org/3/search/movie?api_key=" + tmdb_key + "&query="
        
response_tmdb_id = []

# DON'T FORGET TO DOUBLE CHECK THE YEAR!!!!!
for movie in movies:
    movie_data = requests.get(url_tmdb_id + movie + "&y=2019").json()
    response_tmdb_id.append(movie_data['results'][0]['id'])



In [12]:
response_tmdb_id

[504562,
 399579,
 287947,
 442065,
 327331,
 157433,
 506876,
 587693,
 1578,
 580632,
 618008,
 597588,
 585839,
 599975,
 603702]

# Now take ID #s and run through TMDB

# ***** FIGURE OUT WHAT WE NEED *****

In [13]:
url_tmdb_movie = "https://api.themoviedb.org/3/movie/"



TMDB_df = pd.DataFrame(response_tmdb_id,columns=['ID'],dtype=object)

response_tmdb_movie_info = []

for index, row in TMDB_df.iterrows(): 
    movie_data = requests.get(url_tmdb_movie + str(TMDB_df.ID[index]) + "?api_key=" + tmdb_key).json()
    response_tmdb_movie_info.append(movie_data)

In [14]:
response_tmdb_movie_info

[{'adult': False,
  'backdrop_path': '/xMHeU84wLQ9uH9acJ0oPgdii9wO.jpg',
  'belongs_to_collection': None,
  'budget': 26000000,
  'genres': [{'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}],
  'homepage': 'https://www.motherlessbrooklynfilm.com',
  'id': 504562,
  'imdb_id': 'tt0385887',
  'original_language': 'en',
  'original_title': 'Motherless Brooklyn',
  'overview': 'New York City, 1957. Lionel Essrog, a private detective living with Tourette syndrome, tries to solve the murder of his mentor and best friend, armed only with vague clues and the strength of his obsessive mind.',
  'popularity': 53.506,
  'poster_path': '/jjMJy5OxJHpFIaboCklDIYdcfpD.jpg',
  'production_companies': [{'id': 2297,
    'logo_path': None,
    'name': 'Class 5 Films',
    'origin_country': 'US'},
   {'id': 36209,
    'logo_path': '/doQEl9S1rg0l0NEtpkxf0N46OHQ.png',
    'name': 'MWM Studios',
    'origin_country': 'US'}],
  'production_countries': [{'iso_3166_1': 'US',
    'name': 'United State